### Use LightGBM

In [1]:
from lightgbm import LGBMClassifier

random_state = 42

model = LGBMClassifier(random_state=random_state)
model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

LightGBM의 주요 hyperparameter는 다음과 같습니다.

**boosting_type**

LightGBM은 4가지의 부스팅 타입을 제공합니다. default값은 gbdt (Gradient Boosting Decision Tree)이며
[dart](https://arxiv.org/abs/1505.01866), [goss](https://papers.nips.cc/paper/6579-gradient-based-sampling-an-adaptive-importance-sampling-for-least-squares.pdf), rf (random forest)까지 지원을 합니다.

부스팅 알고리즘의 구체적인 원리를 아는 것은 수업의 범위를 다소 벗어나기 때문에, 간략하게만 요약하자면 다음과 같습니다.

  * gbdt가 가장 일반적으로 이야기하는 Gradient Boosting Machine 알고리즘입니다.
  * rf는 Random Forest 알고리즘입니다.
  * dart는 gbdt의 개선 버전인데, 성능이 더 좋지만 실행 속도가 느립니다.
  * goss도 마찬가지로 gbdt의 개선 버전인데, DART와는 반대로 실행 속도가 빠르지만 gbdt보다 좋은 성능을 내지 않습니다.

**n_estimators**

부스팅에서 사용할 트리의 갯수를 지정해줍니다. 기본 값은 100입니다. 일반적으로 높으면 좋습니다만, 너무 과하면 부스팅 모델이 학습 데이터에 너무 최적화되는 과적화 현상(overfitting)이 일어날 수 있습니다.

**learning_rate**

각각의 부스팅 트리가 결과에 미치는 비율을 조정해줍니다. 기본적으로 0보다 큰 숫자를 적용하되, 너무 크면 각 나무들이 오류를 크게 보정함으로써 성능이 오히려 안 좋아지는 효과가 있습니다. learning_rate가 너무 작을 경우, 상대적으로 n_estimators를 키워야 해서 학습이 오래 걸리는 문제가 있으니 주의해야 합니다. // 1/10, 10배 단위로 tuning

**num_leaves**

각 트리가 최대 몇 개의 leave를 가질 수 있는지 지정할 수 있습니다. 1보다 큰 숫자를 지정하며, 기본값은 31입니다. 보통 scikit-learn 같은 툴에서는 max_depth(나무의 깊이)를 조정함으로써 트리가 overfitting 되는 현상을 막아야 하는데, LightGBM에서는 num_leaves가 이를 대신하고 있습니다. (그렇다고 LightGBM에 max_depth가 없는 건 아니지만, num_leaves에 비해 큰 효과가 있지는 않습니다)

이 값이 크면 나무가 데이터를 더 디테일하게 분석하는 장점이 있지만(=성능이 올라갑니다), 이게 너무 커지면 모델이 overfitting되는 현상이 있으니 주의해야 합니다. 정 반대로 이 값이 너무 작으면 분석력이 떨어져서 모델의 성능이 나빠지게 됩니다.

**max_bin**

하나의 feature에 여러 종류의 값이 있는 경우, 이를 전부 탐색하지 않고 [histogram](https://en.wikipedia.org/wiki/Histogram)처럼 구역(bin)을 나눠서 탐색하는 기능입니다. 주로 연속적인(continuous)한 값에 효과를 발휘합니다. (자세한 사항은 [다음의 링크](https://github.com/Microsoft/LightGBM/issues/1144)를 참고해주세요) 기본 값은 255입니다.

가령 어떤 feature가 1, 2, 3, 4, 5, 6 라는 6종류의 값을 가질 때

  * max_bin이 6이거나 그 이상이면 1, 2, 3, 4, 5, 6 전부를 탐색합니다.
  * 하지만 max_bin이 3라면 (1, 2), (3, 4), (5, 6)을 묶어서 세 그룹으로만 나눠서 탐색합니다. 

이 값이 클수록 연속적인 값을 더 디테일하게 탐색할 수 있습니다. (=성능이 올라갑니다) 하지만 이 값이 너무 크면 알고리즘의 속도가 느려지며, 또한 모델이 overfitting되는 현상이 발생되기 때문에 주의해야 합니다. 정 반대로 모델이 overfitting되는 현상이 발생한다면, 이 max_bin을 줄여줌으로서 overfitting 현상을 완화시킬 수 있습니다.

**min_child_samples**

나무의 잎(leaf)에 적어도 몇개의 데이터가 들어가야 하는지 지를 지정해줍니다. 이 숫자가 너무 작으면 나무가 가지를 너무 많이 치게 되고, 결과적으로 모델이 overfitting 되는 현상이 있습니다. 정 반대로 숫자가 너무 크면 모델의 분석력이 떨어져서 성능이 잘 나오지 않습니다. 그러므로 적당한 값을 찾아서 넣어주는 것이 좋습니다.

**min_split_gain**

나무가 가지를 칠 때, loss(ex: gini impurity, mse, etc)가 min_split_gain 이상으로 줄어들지 않는다면 가지를 치지 않습니다.

가령 가지를 치기 이전에 gini impurity가 0.45, 가지를 친 후의 gini impurity가 0.42라고 한다면, min_split_gain가 0.2이면 가지를 치지만 0.4면 가지를 치지 않습니다. (즉, gini impurity(before) - gini impurity(after) > min_split_gain 일 경우에만 가지를 칩니다)

이 hyperparameter의 기본 값은 0 입니다. 이 값을 늘려주면 결과적으로 가지를 덜 치기 됨으로써 나무가 overfitting되는 현상을 방지할 수 있습니다. 정 반대로 숫자가 너무 크면 가지를 너무 덜 치게 되어 모델의 분석력이 떨어지는 현상이 생깁니다. 마찬가지로 적절한 값을 찾아서 넣어주는 것이 좋습니다.

**subsample**

모델을 학습할 때 얼만큼의 데이터를 사용할지 결정해주는 hyperparameter입니다. 가령 1.0은 데이터를 100% 사용한다는 의미이며, 0.8의 경우 80%만 사용한다는 의미로 받아들이면 됩니다. (기본값은 1.0입니다)

이 기능은 마치 Random Forest의 Bagging(Boostrap Aggregation)과 동일하다고 생각할 수 있지만, 미묘하게 다른 방식입니다. ([참고 링크](https://github.com/Microsoft/LightGBM/issues/883) 하지만 동일한 역할을 수행한다고 보시면 됩니다.

**subsample_freq**

부스팅 알고리즘이 새로운 트리를 만들 때, 몇 번 반복했을 때 subsampling을 다시 할지 결정하는 hyperparameter입니다. 기본값이 0인데, 이는 한 번 subsampling을 하면 학습이 끝날 때 까지 다시 subsampling를 하지 않겠다는 의미입니다. 하지만 10을 넣으면 10번, 20을 넣으면 트리가 20번 생성될 때 마다 매 번 새로운 데이터를 subsampling 합니다.

이 숫자는 (0이 아니라는 전제 하에) 낮을 수록(=1에 가까울수록) 전체 모델의 성능이 좋아집니다. (트리가 다양해지기 때문) 하지만 그만큼 속도가 느리기 때문에 주의해야 할 필요가 있습니다.

**colsample_bytree**

RandomForest의 max_features와 동일한 개념입니다. 즉, 나무를 하나 만들 때 전체 features를 사용하지 않고 일부분을 random sampleing해서 사용합니다. 가령 이 값이 1이면 모든 feature를 사용하며, 0.5이면 전체 feature에서 각 트리마다 랜덤하게 절반만 사용합니다. 이 값이 1보다 작으면 over-fitting을 방지하는 효과가 있습니다. (다만 너무 작으면 트리가 feature를 거의 못 쓰기 때문에 오히려 역효과가 납니다)

In [2]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(boosting_type='gbdt',
                       n_estimators=100,
                       learning_rate=0.1,
                       num_leaves=31,
                       max_bin=255,
                       min_child_samples=20,
                       min_split_gain=0.0,
                       subsample=1.0,
                       subsample_freq=0,
                       colsample_bytree=1.0,
                       n_jobs=-1,
                       random_state=random_state)
model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_bin=255,
        max_depth=-1, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
        objective=None, random_state=42, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)

## 기타

LightGBM에는 이 외에도 많은 Hyperparameter가 있습니다. (다만 이는 필수가 아닙니다)

LightGBM의 Hyperparameter를 더 자세히 알고 싶은 분들은 [다음의 링크](https://lightgbm.readthedocs.io/en/latest/Python-API.html#scikit-learn-api)를 참고해주세요. 앞의 링크 외에도, [이 링크](https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc)가 Hyperparameter에 대해서 자세히 설명하고 있으니 참고하면 좋습니다.

